In [ ]:
import os
import json
import time
import random
import pickle
import collections
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob

from image_captioning.constants import DATA_DIR
from image_captioning.data_pipeline import input_dataset
from image_captioning.model import text_vectorization, encoder, decoder

# Download data

This can take some time as it's ~13GB of data.

In [ ]:
# Download caption annotation files
if not os.path.exists(DATA_DIR):
    annotation_zip = tf.keras.utils.get_file(
        fname='captions.zip',
        origin='http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
        cache_subdir=DATA_DIR,
        extract=True
    )
    os.remove(annotation_zip)

annotation_file = os.path.join(DATA_DIR, 'annotations/captions_train2014.json')

In [ ]:
# Download image files
image_folder = os.path.join(DATA_DIR, 'train2014')
if not os.path.exists(DATA_DIR):
    image_zip = tf.keras.utils.get_file(
        fname='train2014.zip',
        origin='http://images.cocodataset.org/zips/train2014.zip',
        cache_subdir=image_folder,
        extract=True
    )
    os.remove(image_zip)

# Load data

In [ ]:
with open(annotation_file, 'r') as f:
    annotations = json.load(f)

In [ ]:
# Group all captions together having the same image ID.
image_path_to_caption = collections.defaultdict(list)
for val in annotations['annotations']:
    caption = f"<start> {val['caption']} <end>"
    image_path = os.path.join(image_folder, 'COCO_train2014_{:012d}.jpg'.format(val['image_id']))
    image_path_to_caption[image_path].append(caption)

image_paths = list(image_path_to_caption.keys())

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

random_image_path = np.random.choice(image_paths)
ax.set_title("\n".join(image_path_to_caption[random_image_path]), fontsize=14)
ax.imshow(plt.imread(random_image_path));

# Data preprocess image

In [ ]:
random_image_path = np.random.choice(image_paths)
preprocessed_img, _ = input_dataset.load_and_preprocess_image(random_image_path)

print(preprocessed_img.shape)
plt.imshow((preprocessed_img + 1) / 2);

# Encode images

In [ ]:
train_captions = []
img_name_vector = []

for image_path in image_paths:
    caption_list = image_path_to_caption[image_path]
    train_captions.extend(caption_list)
    img_name_vector.extend([image_path] * len(caption_list))  # duplicate image path so that every caption has its own image path

In [ ]:
# initialize inceptionV3 network with imagenet weights
inceptionV3 = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

In [ ]:
os.makedirs(os.path.join(DATA_DIR, 'train2014_features'), exist_ok=True)  # directory where to store processed features

# get unique image paths that are not processed yet
img_paths_processed = [p.replace('_features', '').replace('.npy', '.jpg') for p in glob(os.path.join(image_folder + '_features', '*'))]
encode_train = sorted(set(img_name_vector) - set(img_paths_processed))
print(f"Number of images left to process: {len(encode_train)}")

if len(encode_train) > 0:
    # create dataset that returns images and their corresponding filepaths
    image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
    image_dataset = image_dataset.map(input_dataset.load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    image_dataset = image_dataset.batch(32)

    for batch_imgs, batch_paths in tqdm(image_dataset):

        batch_features = inceptionV3(batch_imgs)  # output shape (bs, 8, 8, 2048)
        batch_features = tf.reshape(batch_features, (batch_features.shape[0], -1, batch_features.shape[3]))  # output shape (bs, 64, 2048); basically flattens the spatial dimension

        for bf, p in zip(batch_features, batch_paths):
            img_path = p.numpy().decode("utf-8")
            filename = img_path.split('/')[-1].split('.')[0]
            output_path = os.path.join(DATA_DIR, 'train2014_features', filename)  # store features in separate directory
            np.save(output_path, bf.numpy())  # will append .npy to output filename

# Preprocess and tokenize captions

In [ ]:
max_length = 50  # max word count for a caption
vocabulary_size = 5000  # use the top 5000 words for a vocabulary

In [ ]:
%%time

tokenizer_path = os.path.join(DATA_DIR, 'experiment/tokenizer.pkl')

if os.path.exists(tokenizer_path):
    tokenizer = text_vectorization.load_text_vectorizer(tokenizer_path)
else:
    tokenizer = text_vectorization.fit_text_vectorizer(train_captions, text_vectorization.standardize_text, max_length, vocabulary_size)
    text_vectorization.save_text_vectorizer(tokenizer, tokenizer_path)

In [ ]:
# Create mappings for words to indices and indicies to words.
word_to_index = tf.keras.layers.StringLookup(mask_token="", vocabulary=tokenizer.get_vocabulary())
index_to_word = tf.keras.layers.StringLookup(mask_token="", vocabulary=tokenizer.get_vocabulary(), invert=True)

# Split data into train/test

In [ ]:
len(train_captions), len(img_name_vector)

In [ ]:
unique_images = list(set(img_name_vector))
np.random.shuffle(unique_images)

slice_index = int(len(unique_images) * 0.8)

img_name_train = []
cap_train = []
for imgt in unique_images[:slice_index]:
    
    filename = imgt.split('/')[-1].replace('.jpg', '.npy')
    features_path = os.path.join(DATA_DIR, 'train2014_features', filename)  # store features in separate directory

    capt_len = len(image_path_to_caption[imgt])
    img_name_train.extend([features_path] * capt_len)
    cap_train.extend(image_path_to_caption[imgt])

img_name_val = []
cap_val = []
for imgv in unique_images[slice_index:]:
    
    filename = imgt.split('/')[-1].replace('.jpg', '.npy')
    features_path = os.path.join(DATA_DIR, 'train2014_features', filename)  # store features in separate directory
    
    capv_len = len(image_path_to_caption[imgv])
    img_name_val.extend([features_path] * capv_len)
    cap_val.extend(image_path_to_caption[imgv])

In [ ]:
len(img_name_train), len(cap_train), len(img_name_val), len(cap_val)

# TF dataset for training

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 1000
embedding_dim = 256
units = 512
num_steps = len(img_name_train) // BATCH_SIZE
# Shape of the vector extracted from InceptionV3 is (64, 2048)
# These two variables represent that vector shape
features_shape = 2048
attention_features_shape = 64

In [ ]:
# Load the numpy files
def map_func(img_name, cap):
    img_tensor = np.load(img_name.decode('utf-8'), allow_pickle=True)
    return img_tensor, cap

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))

# Use map to load the numpy files in parallel
dataset = dataset.map(lambda path, text: (path, tokenizer(text)))

dataset = dataset.map(
    lambda path, text: tf.numpy_function(map_func, [path, text], [tf.float32, tf.int64]), 
    num_parallel_calls=tf.data.AUTOTUNE
)

# Shuffle and batch
dataset = dataset \
    .shuffle(BUFFER_SIZE) \
    .batch(BATCH_SIZE) \
    .prefetch(buffer_size=tf.data.AUTOTUNE)

# Model

In [ ]:
cnn_encoder = encoder.CNN_Encoder(embedding_dim)
rnn_decoder = decoder.RNN_Decoder(embedding_dim, units, tokenizer.vocabulary_size())

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')


def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

# Checkpoint

In [ ]:
checkpoint_path = os.path.join(DATA_DIR, 'experiment/checkpoints/train')
ckpt = tf.train.Checkpoint(encoder=cnn_encoder,
                           decoder=rnn_decoder,
                           optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [ ]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
    # restoring the latest checkpoint in checkpoint_path
    ckpt.restore(ckpt_manager.latest_checkpoint)

# Training

In [ ]:
# adding this in a separate cell because if you run the training cell many times, the loss_plot array will be reset
loss_plot = []

In [ ]:
@tf.function
def train_step(img_tensor, target):
    loss = 0

    # initializing the hidden state for each batch
    # because the captions are not related from image to image
    hidden = rnn_decoder.reset_state(batch_size=target.shape[0])

    dec_input = tf.expand_dims([word_to_index('<start>')] * target.shape[0], 1)

    with tf.GradientTape() as tape:
        features = cnn_encoder(img_tensor)

        for i in range(1, target.shape[1]):
            # passing the features through the decoder
            predictions, hidden, _ = rnn_decoder(dec_input, features, hidden)

            loss += loss_function(target[:, i], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(target[:, i], 1)

    total_loss = (loss / int(target.shape[1]))

    trainable_variables = cnn_encoder.trainable_variables + rnn_decoder.trainable_variables

    gradients = tape.gradient(loss, trainable_variables)

    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss, total_loss

In [ ]:
EPOCHS = 20

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            average_batch_loss = batch_loss.numpy()/int(target.shape[1])
            print(f'Epoch {epoch+1} Batch {batch} Loss {average_batch_loss:.4f}')
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    if epoch % 5 == 0:
        ckpt_manager.save()

    print(f'Epoch {epoch+1} Loss {total_loss/num_steps:.6f}')
    print(f'Time taken for 1 epoch {time.time()-start:.2f} sec\n')

In [ ]:
plt.plot(loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Plot')